# Practical 06 - Regression


As usual, you should save a copy of this notebook in Google drive (or on your own system if not using Colaboratory)

In this session we will:
- load and explore a data set from astronomy
- Perform feature selection, feature engineering, and data scaling
- Train three models on our data:
  - A KNN regressor
  - A linear regression model
  - A descision tree regressor
- Compare and contrast the results from our models.

In [ ]:
# data reading and manipulation libraries
import numpy as np
import pandas as pd
# plotting libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# Machine learning tools
import sklearn
from sklearn import (metrics, preprocessing, linear_model, model_selection)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
print(f"Numpy:        {np.__version__}")
print(f"Pandas:       {pd.__version__}")
print(f"Matplotlib:   {matplotlib.__version__}")
print(f"Seaborn:      {sns.__version__}")
print(f"Scikit-learn: {sklearn.__version__}")

## Data preparation

We will be working with a data set that comes from astronomy.
The data have been downloaded from an archive filtered to be relevant for this practical.
We can pull the data directly from my GitHub.

In [ ]:
galaxy_df = pd.read_csv("https://raw.githubusercontent.com/PaulHancock/2023-ASA-ML-DeepDive/main/SDSS_10k_Galaxy.csv")

In [ ]:
galaxy_df.columns

Above we can see the following columns:
- objid: the object identifier, an integer
- ra, dec: the location of the object on the sky (ra/dec = lat/long for the sky)
- u,g,r,i,z: photometric observations. How bright an object is in each part of the spectrum, measured in magnitudes, which is a logarithmic scale.
- run, rerun, camcol, field, specobjid, plate, fiberid: meta-data associated with the observation. Not specific to the object but descirbes the setup of the instrument during the observaiton.
- redshift: the redshift of the object being observed. This is a measure of distance for astronomers.
- mjd: the date of the observaiton
-SpType, BV, TEff, FeH: properties of the object which are all null because these are only valid for stars and our data has been filtered to include only galaxies.

In [ ]:
galaxy_df.describe()

### Feature selection

We will remove all the "non-physical" properties from our dataset, and our goal is to use the (u,g,r,i,z) photometric magnitudes to predict the redshift of the galaxies.

In astronomy, the redshift is measured by taking a spectrum of an object, and this typically takes a long time to measure using large and very expensive telescopes. The photometric magnitude of an object can be measured relatively quickly, so being able to predict redshifts from photometry means that we could use a lot less time, or by using smaller and less expensive instruments.

In [ ]:
galaxy_df = galaxy_df[ ? ]

### Data scaling

Our data are calibrated to a physically meaningful scale, however the different magnitudes don't share the same distributions.

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(12, 8))
galaxy_df.hist(grid=True, ax=ax)
plt.show()

In order to eliminate issues with scaling we will transform all our features using a min/max scaler.
Some of the features look like they may have a normal or power-law distribution, so min/max might not be the best solution, but this is what we'll go with for now.
You can come back and try different scalers at a later time.

The preprocessing classes have a similar interface to the ML models:
- `.fit` to fit the transformer to your data
- `.transform` to return the transformed data
- `.fit_transform` to do both at once

In [ ]:
# create a minmax scaler
minmax = preprocessing.MinMaxScaler()
# "fit" the scaler -> determine the min/max for each column
minmax.fit(galaxy_df)

#make a copy of the data frame we are about to scale
scaled_df = galaxy_df.copy()
# Transform the data and save it back to our dataframe
scaled_df[:] = minmax.transform(scaled_df)
#        ^ this slicing hack causes the values of the pandas dataframe to
# be overwritten with the values from a numpy array (from the tranform)

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Look at our scaled data to check the effects of our code.

You should see all columns have a $min = 0$ and $max = 1.0$

In [ ]:
scaled_df.describe()

A more compact way to visualise the spread of data within our range $[0,1]$ is to use a box and whisker plot.

Seaborn gives us a fairly nice way to do this, we just have to make sure that we fiddle the axis labels so they don't overlap.

In [ ]:
ax = sns.boxplot(scaled_df)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

For a full view of the correlation of all parameters we can use the pandas `.corr()` method to compute the correlation and then the seaborn `.heatmap()` method to display it.

In [ ]:
# Compute the correlation between each of the numeric parameters as well as the
# target attribute
cor = scaled_df.corr()

# use seaborn to do the plot
fig, ax = plt.subplots(1,1, figsize=(12,12))
sns.heatmap(cor, annot=True, cmap=plt.cm.rainbow, ax=ax)
plt.show()

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Is correlation the best way to judge the utility of a feature?

What assumptions are we making here?

How could we make different assumptions?

### Feature engineering

A commonly used feature that astronomers use to classify galaxies into different types are the spectral shapes (or colours) of the galaxy.
These colours are ratios of brightness levels, which correspond to linear combinations of magnitudes.

Let us add some additional features which are the pariwise differences between neighbouring bands.

Note that we need to compute the difference in the raw data first so that we are working with physically meaningful features. We then scale this with our minmax scaler.

In [ ]:
X = scaled_df.drop(columns='redshift')
# Create colours as our new features
X['u-g'] = galaxy_df['u'] - galaxy_df['g']
X['g-r'] = ?
X['r-i'] = ?
X['i-z'] = ?
# Once computed we can scale the new features so that they also have a range of [0,1]
X[ ['u-g','g-r','r-i','i-z'] ] = minmax.fit_transform( ? )

y = galaxy_df['redshift'] # we want unscaled reshifts to be predicted

In [ ]:
# Compute the correlation between each of the numeric parameters as well as the
# target attribute
cor = X.?

# use seaborn to do the plot
fig, ax = plt.subplots(1,1, figsize=(12,12))
sns.heatmap(cor, annot=True, cmap=plt.cm.rainbow, ax=ax)
plt.show()

Looking at the correlation map above we can see examples of:
- attribtes which have a correlation of ~1,
- attributes which are highly correlated with the redshift,
- attributes which have very low correlation with the redshift.

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

In each of the above cases, how does this inform our feature selection?

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Create a test/train split from our `X` data.


In [ ]:
# Split our data into a test/train set
X_train, X_test, y_train, y_test = ?

## KNN regressor

We can do our gridsearch as before however we can no longer use 'accuracy' as a metrics, since we don't have categorical data.

Instead we need to use a different scoring metric.

For now let's use the $R^2$ metric since it's easy do understand.

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Perform a gridsearch with cross validation using the `r2` metric for scoring.



In [ ]:
# Create a dictionary of all the parameters we'll be iterating over
parameters = {'weights': (?), # this should be the different weighting schemes
              'n_neighbors':[?]} # this should be a list of the nearest neigbhours
# make a classifier object
estimator = ?
# create a GridSearchCV object to do the training with cross validation
gscv = model_selection.GridSearchCV(estimator=?,
                                    param_grid=parameters,
                                    scoring=?)
# now train our model
knn = ?
knn_y_pred = ?
print(knn.best_params_, knn.best_score_)

We have used cross validation which means that we can see the average and variance of the test score across the different validation splits. We can also view this for each of the parameter combinations.

In [ ]:
print("n_neighbors |  weights    |     R2 +/- std")
for i in range(len(knn.cv_results_['params'])):
  print(f"{knn.cv_results_['params'][i]['n_neighbors']:4d}   ", end='')
  print(f"{knn.cv_results_['params'][i]['weights']:>15s}     ", end='')
  print(f"{knn.cv_results_['mean_test_score'][i]: 5.3f}  +/- ", end='')
  print(f"{knn.cv_results_['std_test_score'][i]:5.3f}")

Is there a significant difference between the best and worst performing model in the above table?

How do you interpret your answer to the above?

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Retrain our KNN model using the best parameters and recompute the $R^2$ metric. Compare the new score to our expecations from above.

In [ ]:
# use the best n_neighbors and weights
knn = ?
knn.fit(X_train, y_train)
y_pred = ?
r2 = metrics.r2_score(?)
print(f'R^2 = {r2}')

We are going to be running more than one model so let us save a summary of the results into a dataframe.

In [ ]:
deltas = pd.DataFrame()
deltas['y-knn_y'] = y_test-knn_y_pred
deltas.describe()

The above data frame doesn't give us a good feeling of how well the model predicts the data, so let's make a plot to visualise it.

In [ ]:
mx = max(y_test.max(), knn_y_pred.max())
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(y_test,knn_y_pred, 'b.')
ax.plot([0,mx], [0, mx], 'k--')
ax.set_xlabel("true")
ax.set_ylabel("predicted")
ax.set_title("KNN redshift estimation")
plt.show()

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)
In the above plot, you should see that your ML model has done a fairly good job of estimating the redshift for the vast majority of galaxies.
The variance is less than 0.1.
However, the predictor doesn't seem to want to predict redshifts higher than about 1.0, even though there are some $z~1$ galaxies in our sample.

Why might this be the case?

## Linear regression

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Fit a linear regression model (`linear_model.LinearRegression` to our data and predict some redshifts.

In [ ]:
# no need for a grid search as there are no hyper-parameters to tune
lr = ?
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)

For a little fun, let's have a look at the euqation of best fit that was generated by our model.

In [ ]:
print("redshift = ",end='')
for feature, coef in zip(X.columns, lr.coef_):
  print(f"{coef:+5.3f}*{feature} ", end='')
print(f"{lr.intercept_:+5.3f}")

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

What does it mean for a feature to have a high or low weight in the above equation?

If we were using un-scaled data would you have a different answer?

In [ ]:
mx = max(y_test.max(), lr_y_pred.max())
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(y_test,lr_y_pred, 'b.')
ax.plot([0,mx], [0, mx], 'k--')
ax.set_xlabel("true")
ax.set_ylabel("predicted")
ax.set_title("Linear Regression redshift estimation")
plt.show()

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

What are some features of the above plot that are different from the KNN version of the plot?

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Add a new column to our `deltas` data frame for the linear regression results and compare them to the knn results.

In [ ]:
deltas['y-lr_y'] = ?
deltas.describe()

## Regression trees

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Train a `DescisionTreeRegressor` with a range of `max_depth` and `min_samples_split`, using the `r2` scoring metric.

In [ ]:
# Create a dictionary of all the parameters we'll be iterating over
parameters = {'max_depth': [?], # choose a range of values from 2 to 20
              'min_samples_split':[ ? ]}
# make a classifier object
?
# create a GridSearchCV object to do the training with cross validation
?
# now train our model
dtr = ?
dtr_y_pred = ?
print(dtr.best_params_, dtr.best_score_)

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Use the best parameters from above to retrain on all the training data.

In [ ]:
best_dtr = DecisionTreeRegressor(max_depth=7,
                                 min_samples_split=1000)
best_dtr.fit(X_train, y_train)
dtr_y_pred = best_dtr.predict(X_test)

As with our classifier trees, we can ask for a visual representation of the tree.
This time the colouring of the nodes represents the value assigned to each group, not the purity of the group.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(18,12))
sklearn.tree.plot_tree(best_dtr,
               filled=True,
               feature_names = X.columns,
               rounded=True,
               ax=ax,
               fontsize=12, )
plt.show()

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Inspect the tree above and determine how many unique values of redshift it can produce.

Additionally, determine if there are any features that are not used, and review these features in the correlation plot we made earlier. Do you see any agreement?

In [ ]:
deltas['y-dtr_y'] = ?
deltas.describe()

In [ ]:
mx = max(y_test.max(), dtr_y_pred.max())
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(y_test,dtr_y_pred, 'b.')
ax.plot([0,mx], [0, mx], 'k--')
ax.set_xlabel("true")
ax.set_ylabel("predicted")
ax.set_title("Descision Tree Regression redshift estimation")
plt.show()

![Challenge](https://cdn.icon-icons.com/icons2/2110/PNG/64/challenge_icon_131034.png)

Based on the graph, the $R^2$ metric, and the above table, which of the three models is performing the best?